In [ ]:
!pip install datasets
!pip install evaluate
!pip install peft
!pip install wandb

In [ ]:
from datasets import load_dataset, Dataset, concatenate_datasets, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_scheduler, DataCollatorWithPadding, Trainer, TrainingArguments
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader
from torch.optim import AdamW
import torch
import torch.nn.functional as F
from tqdm.auto import tqdm
import evaluate
import pandas as pd
from peft import get_peft_model, LoraConfig, TaskType
import numpy as np
from collections import Counter
import gc

In [ ]:
# hyperparameters
lr = 1e-4
num_epochs = 10 # 4 (or 7 epochs seems good)
batch_size = 16
llama_checkpoint = "meta-llama/Llama-2-7b-hf"
# MAX_LEN = 512  # mandotory for roberta optional for others


In [ ]:
llama_model = AutoModelForSequenceClassification.from_pretrained(
    pretrained_model_name_or_path=llama_checkpoint,
    num_labels=8,
    # device_map="auto", #should try without this
    offload_folder="offload",# try without this
    trust_remote_code=True
)
#For Llama 2, we have to add the padding token id as it is not defined by default.
llama_model.config.pad_token_id = llama_model.config.eos_token_id


config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-2-7b-hf and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
llama_tokenizer = AutoTokenizer.from_pretrained(llama_checkpoint, add_prefix_spac=True)
llama_tokenizer.pad_token_id = llama_tokenizer.eos_token_id
llama_tokenizer.pad_token = llama_tokenizer.eos_token


tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
train_data_files = {
    "train": "MaSaC_train_erc_task1.json",
    "validation": "MaSaC_validation_erc_task1.json"
}

train_dataset = load_dataset("json", data_files=train_data_files)
train_dataset = train_dataset.remove_columns("episode")

labels = ['anger', 'neutral', 'contempt', 'sadness', 'fear', 'disgust', 'joy', 'surprise']
label_encoder = LabelEncoder()
label_encoder.fit(labels)

combined_dataset = {}
for dataset_type in train_dataset:
    for record in train_dataset[dataset_type]:
        encoded_label = label_encoder.transform(record["emotions"])
        record["emotions"] = encoded_label
        if combined_dataset.get(dataset_type) is None:
            combined_dataset[dataset_type] = Dataset.from_dict(record)
        else:
            combined_dataset[dataset_type] = concatenate_datasets([combined_dataset[dataset_type], Dataset.from_dict(record)])

dataset_dict = DatasetDict(combined_dataset)


dataset_dict['train'].to_pandas().info()
dataset_dict['validation'].to_pandas().info()

max_char = dataset_dict['train'].to_pandas()['utterances'].str.len().max()
max_words = dataset_dict['train'].to_pandas()['utterances'].str.split().str.len().max()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8506 entries, 0 to 8505
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   emotions    8506 non-null   float64
 1   speakers    8506 non-null   object 
 2   utterances  8506 non-null   object 
dtypes: float64(1), object(2)
memory usage: 199.5+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1354 entries, 0 to 1353
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   emotions    1354 non-null   float64
 1   speakers    1354 non-null   object 
 2   utterances  1354 non-null   object 
dtypes: float64(1), object(2)
memory usage: 31.9+ KB


code above common for all 3 models

In [ ]:
label_counts = Counter(dataset_dict["train"]["emotions"])
num_classes = 8
num_samples = len(dataset_dict["train"])
class_weights = [num_samples / (num_classes * label_counts[i]) for i in range(num_classes)]

In [ ]:
col_to_delete = ["speakers", "utterances"]

def llama_preporcessing_function(exampmle):
    return llama_tokenizer(exampmle["utterances"], truncation=True)

llama_tokenized_datasets = dataset_dict.map(llama_preporcessing_function, batched=True, remove_columns=col_to_delete)
llama_tokenized_datasets = llama_tokenized_datasets.rename_column("emotions", "label") # check the label val for each model individually
llama_tokenized_datasets.set_format("torch")

llama_data_collator = DataCollatorWithPadding(tokenizer=llama_tokenizer)


Map:   0%|          | 0/8506 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/1354 [00:00<?, ? examples/s]

In [ ]:
# Lora setup for llama classifier
llama_peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS, r=16, lora_alpha=16, lora_dropout=0.5, bias="none",
    target_modules=[
        "q_proj",
        "v_proj"
    ]
)
llama_model = get_peft_model(llama_model, llama_peft_config)
llama_model.print_trainable_parameters()


trainable params: 8,421,376 || all params: 6,615,797,760 || trainable%: 0.1272919201206054


In [ ]:
def compute_metrics(eval_pred):
    precision_metric = evaluate.load("precision")
    recall_metric = evaluate.load("recall")
    f1_metric = evaluate.load("f1")
    accuracy_metric = evaluate.load("accuracy")

    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision = precision_metric.compute(predictions=predictions, references=labels, average='weighted')["precision"]
    recall = recall_metric.compute(predictions=predictions, references=labels, average='weighted')["recall"]
    f1 = f1_metric.compute(predictions=predictions, references=labels, average='weighted')["f1"]
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"]
    return {"precision": precision, "recall": recall, "f1-score": f1, 'accuracy': accuracy}


In [ ]:
torch.cuda.empty_cache()
gc.collect()


91

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [ ]:
# not that good at all, wasnt representative
class WeightedCELossTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        # Get model's predictions
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # Compute custom loss
        loss_fct = torch.nn.CrossEntropyLoss(weight=torch.tensor(class_weights, device=model.device, dtype=logits.dtype))
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss


In [ ]:
# can make this custom loss function https://chat.openai.com/share/5c3b8a5b-7cbb-4bed-bcff-7470773cfc26
class WeightedCEWithF1LossTrainer(Trainer):

    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # class_weights = torch.tensor(class_weights, device=model.device, dtype=logits.dtype)
        loss = self.weighted_ce_f1_loss(labels, logits, torch.tensor(class_weights, device=model.device, dtype=logits.dtype))

        return (loss, outputs) if return_outputs else loss

    @staticmethod
    def weighted_ce_f1_loss(y_true, logits, epsilon=1e-7):
        # Convert labels to one-hot encoding
        y_true_one_hot = F.one_hot(y_true, num_classes=logits.size(-1)).float()

        # Apply softmax to logits to get probabilities
        y_pred = F.softmax(logits, dim=-1)

        # Calculate Cross Entropy Loss without reduction
        ce = F.cross_entropy(logits, y_true, reduction='none')
        # Ensure ce is [batch_size, 1] for broadcasting
        ce = ce.view(-1, 1)

        # Calculate components of F1 score for each sample and each class
        tp = (y_true_one_hot * y_pred)
        fp = ((1 - y_true_one_hot) * y_pred)
        fn = (y_true_one_hot * (1 - y_pred))

        # Sum tp, fp, fn over all samples for each class
        tp_sum = tp.sum(dim=0)
        fp_sum = fp.sum(dim=0)
        fn_sum = fn.sum(dim=0)

        # Calculate Precision and Recall for each class
        precision = tp_sum / (tp_sum + fp_sum + epsilon)
        recall = tp_sum / (tp_sum + fn_sum + epsilon)

        # Calculate F1 Score for each class
        f1 = 2 * precision * recall / (precision + recall + epsilon)
        f1 = torch.where(torch.isnan(f1), torch.zeros_like(f1), f1)

        # Calculate mean F1 score across all classes
        f1_mean = f1.mean()
        # Ensure f1_mean is broadcastable with ce
        f1_mean = f1_mean.view(1, -1)

        # Calculate Weighted Loss
        weighted_loss = ce * (1 - f1_mean)
        return torch.mean(weighted_loss)


In [ ]:
class WeightedF1ApproxLossTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")

        # Cross-entropy part
        ce_loss_fct = torch.nn.CrossEntropyLoss(weight=torch.tensor(class_weights, device=model.device, dtype=logits.dtype))
        ce_loss = ce_loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))

        # F1-approximating part
        # Softmax to get probabilities
        probs = F.softmax(logits, dim=-1)
        # One-hot encode labels
        true_labels = F.one_hot(labels, num_classes=self.model.config.num_labels).to(dtype=probs.dtype)

        # Calculate precision and recall per class
        tp = torch.sum(probs * true_labels, axis=0)
        fp = torch.sum(probs * (1 - true_labels), axis=0)
        fn = torch.sum((1 - probs) * true_labels, axis=0)

        precision = tp / (tp + fp + 1e-8)
        recall = tp / (tp + fn + 1e-8)

        # Use harmonic mean of precision and recall as a proxy for F1
        f1_proxy = 2 * (precision * recall) / (precision + recall + 1e-8)
        f1_proxy_loss = 1 - torch.mean(f1_proxy)  # Loss is 1 - mean F1 proxy

        # Combine losses
        # You might need to tune the balance coefficient
        balance_coefficient = 0.5
        loss = (1 - balance_coefficient) * ce_loss + balance_coefficient * f1_proxy_loss

        return (loss, outputs) if return_outputs else loss


In [ ]:
llama_model = llama_model.cuda()

In [ ]:
training_args = TrainingArguments(
    output_dir="llama-lora-token-classification",
    learning_rate=lr,
    lr_scheduler_type= "constant",
    warmup_ratio= 0.1,
    max_grad_norm= 0.3,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.001,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="wandb",
    fp16=True,
    gradient_checkpointing=True,
)


In [ ]:
llama_trainer = WeightedCEWithF1LossTrainer(
    model=llama_model,
    args=training_args,
    train_dataset=llama_tokenized_datasets['train'],
    eval_dataset=llama_tokenized_datasets["validation"],
    data_collator=llama_data_collator,
    compute_metrics=compute_metrics
)


In [ ]:
torch.cuda.empty_cache()
gc.collect()


68

In [ ]:
llama_trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


RuntimeError: "nll_loss_forward_no_reduce_cuda_kernel_index" not implemented for 'Float'

Eval

In [ ]:
test_data_files = {
    "test": "MaSaC_test_erc_task1.json",
}

dataset = load_dataset("json", data_files=test_data_files)
dataset = dataset.remove_columns("episode")

combined_dataset = {}
for dataset_type in dataset:
    for record in dataset[dataset_type]:
        if combined_dataset.get(dataset_type) is None:
            combined_dataset[dataset_type] = Dataset.from_dict(record)
        else:
            combined_dataset[dataset_type] = concatenate_datasets([combined_dataset[dataset_type], Dataset.from_dict(record)])

dataset_dict_test = DatasetDict(combined_dataset)
print(dataset_dict_test)


Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    test: Dataset({
        features: ['utterances', 'speakers'],
        num_rows: 1580
    })
})


In [ ]:
llama_tokenized_datasets = dataset_dict_test.map(llama_preporcessing_function, batched=True, remove_columns=col_to_delete)
llama_tokenized_datasets.set_format("torch")


Map:   0%|          | 0/1580 [00:00<?, ? examples/s]

In [ ]:
print(llama_tokenized_datasets)

DatasetDict({
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 1580
    })
})


In [ ]:
test_dataloader = DataLoader(llama_tokenized_datasets["test"], batch_size=15, collate_fn=llama_data_collator)


In [ ]:
emotions = []

In [ ]:
llama_model.eval()
for batch in test_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = llama_model(**batch)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    predictions = predictions.cpu().tolist()
    decoded_label = label_encoder.inverse_transform(predictions).tolist()
    emotions = emotions + decoded_label


In [ ]:
print(emotions, len(emotions))

['sadness', 'neutral', 'neutral', 'neutral', 'anger', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'fear', 'anger', 'neutral', 'neutral', 'sadness', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'anger', 'neutral', 'neutral', 'neutral', 'neutral', 'anger', 'neutral', 'anger', 'sadness', 'joy', 'neutral', 'joy', 'neutral', 'neutral', 'neutral', 'anger', 'anger', 'anger', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'surprise', 'joy', 'joy', 'joy', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'anger', 'joy', 'neutral', 'neutral', 'surprise', 'joy', 'neutral', 'neutral', 'neutral', 'joy', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'contempt', 'neutral', 'neutral', 'neutral', 'neutral', 'anger', 'anger', 'neutral', 'neutral', 'neutral', 'joy', 'joy', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'anger', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutra

In [ ]:
file_path = 'answer_t1_llama_0_43.txt'

with open(file_path, 'w') as file:
    for item in emotions:
        file.write(item + '\n')
